# Create Confusion Matrix
    1. Load dataset(csv)
    2. Split dataset to X_train,X_test,y_train,y_test
    3. Run classifier, by .fit(X_train,y_train)
    4. Impact on the results, by .predict(X_test)
    5. Import confusion_matrix and Create-> cm = confusion_matrix(y_test, y_pred)

In [1]:
import pandas as pd
import numpy as np
import pickle
import xlwt
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import KFold
from sklearn.metrics import precision_score

In [2]:
df_file = pd.read_csv('../data/df_dropSub_less20_dropNaResult.csv',delimiter=",", skip_blank_lines = True, 
                 error_bad_lines=False)
df_file = df_file.drop('Unnamed: 0',axis=1)
df_file = df_file.fillna(0)
df_file = df_file.replace(['A', 'B+', 'B', 'C+', 'C' , 'D+' , 'D' , 'F' , 'W' , 'S' , 'S#' , 'U' , 'U#'], 
                     [8, 7, 7, 6 , 6, 5, 5, 4, 3, 2, 2, 1, 1])

In [3]:
df_file

,Unnamed: 0.1,3COURSEID,4RESULT,0STUDENTID,1ACADYEAR,2SEMESTER,AT316,AT326,BA291,CJ315,...,TA395,TH161,TU100,TU110,TU120,TU122,TU130,TU154,PROVINCEID,SCHOOLGPA
0,0,CS101,6,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
1,1,CS102,6,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
2,2,EL171,5,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
3,3,SC135,4,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
4,4,SC185,6,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
5,5,TH161,6,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
6,6,TU154,5,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
7,7,CS111,5,316644,2552,2,0,0,0,0,...,0,6,0,0,0,0,0,5,12,3.32
8,8,EL172,4,316644,2552,2,0,0,0,0,...,0,6,0,0,0,0,0,5,12,3.32
9,9,MA211,4,316644,2552,2,0,0,0,0,...,0,6,0,0,0,0,0,5,12,3.32


In [4]:
count_courseId = df_file["3COURSEID"].value_counts() 
more20 = count_courseId

headers=list(df_file.columns.values)
subjects = []
countSub = 0

In [5]:
count = 0
subjects.sort()
precision_rf={}
df_precision = more20.drop('CS231').copy()

list_allsub = df_file.columns[4:]
allSubject_df = pd.DataFrame(columns=[subjects],index=[list_allsub])
top10_df = pd.DataFrame(columns=[subjects])

In [6]:
headers=list(df_file.columns.values)
subjects = []
countSub = 0
#Create dictionary of list subjects
for sub in df_file[headers[1]]:
    if sub not in subjects:
        subjects.append(sub)
        countSub = countSub+1

In [7]:
subjects.sort()

In [8]:
subjects.remove('CS231')

In [9]:
len(subjects)

110

In [16]:
subject = 'MA211'
df_sub = df_file[df_file['3COURSEID'] == subject]
df_sub = df_sub.iloc[np.random.permutation(len(df_sub))]
count_enrollment = df_sub['3COURSEID'].value_counts()
#print "Number of %s enrollment: %s"%(subject,count_enrollment)

A = df_sub.as_matrix()
X = A[:,6:116]
X = X.astype(np.int64, copy=False)
y = A[:,2]
y = y.astype(np.int64, copy=False)

In [17]:
subject

'MA211'

In [18]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 8, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [19]:
X.shape

(1068L, 110L)

In [20]:
y

array([3, 6, 3, ..., 6, 3, 5], dtype=int64)

In [21]:
y.shape

(1068L,)

In [232]:
# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=0)


In [233]:
X_train

array([[0, 0, 0, ..., 0, 6, 4],
       [0, 0, 0, ..., 0, 0, 3],
       [0, 7, 0, ..., 7, 6, 5],
       ..., 
       [0, 0, 0, ..., 0, 0, 6],
       [0, 0, 0, ..., 6, 0, 7],
       [0, 0, 0, ..., 0, 0, 7]], dtype=int64)

In [234]:
X_train.shape

(113L, 110L)

In [235]:
X_test

array([[0, 8, 0, ..., 0, 6, 6],
       [0, 0, 0, ..., 0, 0, 8],
       [0, 0, 0, ..., 0, 7, 6],
       ..., 
       [0, 0, 0, ..., 6, 0, 6],
       [0, 0, 0, ..., 0, 0, 6],
       [0, 8, 0, ..., 0, 0, 5]], dtype=int64)

In [236]:
X_test.shape

(29L, 110L)

In [97]:
X_train.shape

(266L, 110L)

In [98]:
y_train

array([8, 8, 7, 8, 8, 8, 8, 7, 8, 8, 8, 6, 8, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 7, 8, 8, 8, 8, 8, 7, 8, 8, 8, 7, 8, 7, 7, 8, 8, 8, 8, 7, 8,
       7, 7, 8, 8, 8, 8, 8, 7, 7, 7, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 8,
       8, 8, 8, 7, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 8, 7, 7, 8,
       8, 7, 8, 8, 7, 7, 8, 8, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7,
       8, 7, 8, 8, 8, 8, 8, 8, 8, 6, 8, 8, 8, 8, 8, 7, 7, 8, 8, 8, 7, 8, 8,
       8, 8, 8, 8, 8, 7, 8, 8, 7, 8, 8, 8, 7, 8, 8, 8, 8, 8, 7, 8, 8, 8, 8,
       8, 8, 7, 7, 8, 7, 8, 8, 8, 7, 8, 7, 8, 8, 7, 7, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 6, 8, 6, 8, 8, 7, 8, 8, 8, 8, 8, 8, 7, 7,
       8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 6, 7, 8, 8, 7, 8, 8, 8, 8, 7, 8, 8,
       8, 6, 8, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8], dtype=int64)

In [99]:
y_train.shape

(266L,)

In [100]:
y_test

array([8, 7, 8, 7, 8, 7, 8, 8, 8, 8, 8, 7, 8, 8, 8, 7, 8, 8, 8, 8, 8, 8, 8,
       8, 7, 7, 7, 7, 8, 7, 8, 8, 8, 7, 7, 8, 7, 8, 7, 6, 8, 8, 8, 7, 6, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], dtype=int64)

In [101]:
y_test.shape

(67L,)

In [18]:
forest = RandomForestClassifier(n_estimators=10, max_depth=None, 
            min_samples_split=1, random_state=None, max_features=None)
clf = forest.fit(X, y)

In [109]:
cm

array([[ 0,  0,  2],
       [ 0,  1, 10],
       [ 0,  4, 49]])

In [104]:
X_train

array([[0, 0, 0, ..., 0, 6, 6],
       [0, 0, 0, ..., 0, 6, 6],
       [0, 7, 6, ..., 0, 6, 5],
       ..., 
       [0, 0, 0, ..., 0, 7, 6],
       [0, 8, 0, ..., 8, 6, 5],
       [0, 7, 0, ..., 5, 6, 6]], dtype=int64)

In [61]:
len(X_train)

855

In [24]:
y_pred = clf.predict(X)

In [25]:
y_pred.shape

(765L,)

In [26]:
# Compute confusion matrix
cm = confusion_matrix(y, y_pred)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()

Confusion matrix, without normalization
[[ 24   2   6   0   0   0]
 [  0  90   9   1   0   0]
 [  1   3 171  18   2   0]
 [  1   2  17 273   2   2]
 [  0   0   4  12  78   1]
 [  0   0   0   2   3  41]]


In [20]:
Grade = ['A', 'B', 'C' , 'D' , 'F' , 'W' , 'S' , 'U' ,'na']
row = []
for cls in y:
    if cls not in row:
        row.append(cls)
row.sort()
print row

row_cm = []
for i in xrange(len(row)):
    Grade = ['A', 'B', 'C' , 'D' , 'F' , 'W' , 'S' , 'U' ,'na']
    grade = Grade[::-1][row[i]]
    print grade
    row_cm.append(grade)
print row_cm

[3, 4, 5, 6, 7, 8]
W
F
D
C
B
A
['W', 'F', 'D', 'C', 'B', 'A']


In [21]:
row_cm

['W', 'F', 'D', 'C', 'B', 'A']

In [22]:
def plot_confusion_matrix(cm, title=subject, cmap=plt.cm.Blues):
   
    for y in range(cm.shape[0]):
        for x in range(cm.shape[1]):
            plt.text(x, y, '%.1f' % cm[y, x])
  
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(row_cm))
    plt.xticks(tick_marks, row_cm, rotation=45)
    plt.yticks(tick_marks, row_cm)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    
#     plt.pcolor(cm, cmap=cmap)
#     plt.yticks(np.arange(len(row_cm)),row_cm,label="#Training Data")
#     plt.xticks(np.arange(len(row_cm)),row_cm,label='#Class')
#     plt.colorbar()


In [34]:
# Compute confusion matrix
cm = confusion_matrix(y, y_pred)
np.set_printoptions(precision=2)
print('Confusion matrix')
print(row_cm)
print(cm)
plt.figure()
plot_confusion_matrix(cm)

Confusion matrix
['W', 'F', 'D', 'C', 'B', 'A']
[[ 24   2   6   0   0   0]
 [  0  90   9   1   0   0]
 [  1   3 171  18   2   0]
 [  1   2  17 273   2   2]
 [  0   0   4  12  78   1]
 [  0   0   0   2   3  41]]


In [35]:
cm

array([[ 24,   2,   6,   0,   0,   0],
       [  0,  90,   9,   1,   0,   0],
       [  1,   3, 171,  18,   2,   0],
       [  1,   2,  17, 273,   2,   2],
       [  0,   0,   4,  12,  78,   1],
       [  0,   0,   0,   2,   3,  41]])

In [36]:
plt.show()


# Confusion Matrix All Subjects
(has people enroll but no grade result)

In [237]:
for subject in subjects:
    #Create new Dataframe
    
    #print subject             
    df_sub = df_file[df_file['3COURSEID'] == subject]
    df_sub = df_sub.iloc[np.random.permutation(len(df_sub))]
    count_enrollment = df_sub['3COURSEID'].value_counts()
    #print "Number of %s enrollment: %s"%(subject,count_enrollment)

    A = df_sub.as_matrix()
    X = A[:,6:117]
    X = X.astype(np.int64, copy=False)
    y = A[:,2]
    y = y.astype(np.int64, copy=False)
    
    forest = RandomForestClassifier(n_estimators=10, max_depth=None, 
            min_samples_split=1, random_state=None, max_features=None)
    # Split the data into a training set and a test set
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=0)

    clf = forest.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    cm = confusion_matrix(y_test, y_pred)
    print('Confusion matrix of %s'%subject)
    print(cm)
    plt.figure()
    
    
    
    
#     # Split the data into a training set and a test set
#     X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=0)
    
#     forest = RandomForestClassifier(n_estimators=10, max_depth=None, 
#             min_samples_split=1, random_state=None, max_features=None)
#     clf = forest.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
    
#     cm = confusion_matrix(y_test, y_pred)
#     print('Confusion matrix of %s'%subject)
#     print(cm)
#     plt.figure()
    
    #plot_confusion_matrix(cm)
    

Confusion matrix of AT316
[[ 0  0  1  0]
 [ 0  0  1  0]
 [ 0  0 18  4]
 [ 0  0  3  2]]
Confusion matrix of AT326
[[ 0  1  0]
 [ 0 23  6]
 [ 0 12  3]]
Confusion matrix of BA291
[[0 0 0 1 0]
 [0 0 1 0 0]
 [0 0 0 4 0]
 [0 0 3 2 0]
 [0 0 1 0 0]]
Confusion matrix of CJ315
[[5 0]
 [1 0]]
Confusion matrix of CJ316
[[0 0 0]
 [1 4 1]
 [0 2 0]]
Confusion matrix of CJ317
[[0 0 0]
 [2 3 1]
 [0 2 0]]
Confusion matrix of CJ321
[[8 0]
 [1 0]]
Confusion matrix of CS101
[[ 0  0  0  1  0  0]
 [ 0  0  3 12  0  0]
 [ 0  1  2 30  0  0]
 [ 0  2 11 84  5  0]
 [ 0  0  1 31  0  0]
 [ 0  1  0  6  0  0]]
Confusion matrix of CS102
[[ 0  0  1  1  0  0]
 [ 0  0  0  9  2  1]
 [ 0  0  5 27  6  1]
 [ 1  0  9 56  9  4]
 [ 0  0  6 27  9  2]
 [ 0  0  1 14  2  1]]
Confusion matrix of CS105
[[ 0  0  8  2  1  0]
 [ 1  0  1  0  0  0]
 [ 1  0  7  1  3  0]
 [ 2  0 10  3  3  0]
 [ 1  0  9  2  1  0]
 [ 1  0  0  2  1  0]]
Confusion matrix of CS111
[[ 2  1  3  0  0  0]
 [ 3  8  7  5  0  0]
 [ 3 10 16 14  1  0]
 [ 3  5 14 30  6  3]